In [23]:
# Data handling
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from math import pi

In [24]:
# Bokeh libraries
from bokeh.io import output_file, output_notebook
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource
from bokeh.layouts import row, column, gridplot
from bokeh.models.widgets import Tabs, Panel
from bokeh.models import ColumnDataSource, HoverTool, CategoricalColorMapper, LinearColorMapper, ColorBar, NumeralTickFormatter
from bokeh.palettes import Category20c, Spectral5, Viridis
from bokeh.transform import cumsum, factor_cmap, transform

In [25]:
# The figure will be rendered inline a my Jupyter Notebook
output_notebook()

Loading BokehJS ...

In [26]:
df = pd.read_csv('../data/dash_fd_salaries_2019.csv')

In [27]:
df.head()

,Week,Year,GID,FirstName,LastName,Pos,Team,h/a,Oppt,fd_points,fd_salary
0,14,2019,1151,Brees,Drew,QB,nor,h,sfo,40.06,7700.0
1,14,2019,1465,Winston,Jameis,QB,tam,h,ind,37.74,7700.0
2,14,2019,1519,Trubisky,Mitchell,QB,chi,h,dal,33.06,7300.0
3,14,2019,1518,Watson,Deshaun,QB,hou,h,den,30.08,7700.0
4,14,2019,1452,Garoppolo,Jimmy,QB,sfo,a,nor,29.06,7800.0


In [28]:
df.describe()

,Week,Year,GID,fd_points,fd_salary
count,1806.000000,1806.0,1806.000000,1806.000000,1538.000000
mean,15.489480,2019.0,4996.930786,286.872171,5448.959688
std,1.114324,0.0,1426.589789,1025.996809,1358.106181
min,14.000000,2019.0,1131.000000,-2.340000,0.000000
25%,14.000000,2019.0,4688.000000,0.800000,4500.000000
50%,15.000000,2019.0,5490.000000,4.800000,5000.000000
75%,16.000000,2019.0,5735.000000,12.415000,6275.000000
max,17.000000,2019.0,7032.000000,5000.000000,11000.000000


In [29]:
# Removing lines with salary = 0
remove_sal = df['fd_salary'] != 0
df = df[remove_sal]

In [30]:
# Removing lines with points negative points
remove_poi = df['fd_points'] >= 0
df = df[remove_poi]

In [31]:
# Removing NaN 
df = df.dropna()

In [32]:
# Creating column Name with first and last name 
df['Name'] = df['LastName'] + ' ' + df['FirstName']

In [33]:
# Deleting first and last name columns
del df['FirstName'] 
del df['LastName']

In [34]:
df.head()

,Week,Year,GID,Pos,Team,h/a,Oppt,fd_points,fd_salary,Name
0,14,2019,1151,QB,nor,h,sfo,40.06,7700.0,Drew Brees
1,14,2019,1465,QB,tam,h,ind,37.74,7700.0,Jameis Winston
2,14,2019,1519,QB,chi,h,dal,33.06,7300.0,Mitchell Trubisky
3,14,2019,1518,QB,hou,h,den,30.08,7700.0,Deshaun Watson
4,14,2019,1452,QB,sfo,a,nor,29.06,7800.0,Jimmy Garoppolo


In [35]:
# Most expensive players
expensive = df.groupby('Name')[['fd_salary']].sum()
expensive = expensive.sort_values(by='fd_salary', ascending=False).head(5)
expensive

,fd_salary
Name,
Christian McCaffrey,43200.0
Michael Thomas,35900.0
Ezekiel Elliott,34200.0
Saquon Barkley,34000.0
Patrick Mahomes II,33700.0


In [36]:
# Plot - Most expensive players

expensive = pd.DataFrame(expensive)
source = ColumnDataSource(expensive)

hover = HoverTool(tooltips = [("Player", "@Name"), ("Salary", '$'"@fd_salary")])

names_list = source.data['Name'].tolist()
p4 = figure(
       y_range = names_list,
       title='MOST EXPENSIVE PLAYERS',
       x_axis_label = 'Salary',
       y_axis_label = '',
       plot_height=500,
       plot_width=800,
       tools=[hover],
       toolbar_location=None,
)

p4.hbar(y='Name', 
       right='fd_salary', 
       height=0.4, 
       fill_color=factor_cmap(
              'Name',
              palette=Viridis[5],
              factors=names_list
       ),
       alpha=0.8,
       source=source
)

p4.title.text_font_size='25px'
p4.title.align='center'
p4.title.text_color='#4D5656'
p4.xgrid.grid_line_color = None
p4.xaxis[0].formatter = NumeralTickFormatter(format="$0")

show(p4)

In [37]:
# Avg points per position
avg_points_per_position = df.groupby('Pos')[['fd_points']].mean()
avg_points_per_position.head()

,fd_points
Pos,
QB,15.692958
RB,6.629551
TE,4.092657
WR,5.810100


In [38]:
# Plot - Avg points per position

x = {
    'Quarter Backs': 15.692958,
    'Running Backs': 6.629551,
    'Tight Ends': 4.092657,
    'Wide Receivers': 5.810100
}

data = pd.Series(x).reset_index(name='value').rename(columns={'index':'Pos'})
data['angle'] = data['value']/data['value'].sum() * 2*pi
data['color'] = Viridis[len(x)]

p5 = figure(
    title='AVG POINTS PER POSITION', 
    plot_height=500,
    plot_width=400, 
    toolbar_location=None,
    tools='hover', 
    tooltips='@Pos: @value', 
    x_range=(-0.5, 1.0),
)

p5.title.text_font_size='25px'
p5.title.align='center'
p5.title.text_color='#4D5656'
p5.xaxis[0].formatter = NumeralTickFormatter(format="$0")

p5.wedge(x=0, y=1, radius=0.4,
        start_angle=cumsum('angle', include_zero=True), 
        end_angle=cumsum('angle'),
        line_color="white", 
        fill_color='color', 
        legend_field='Pos', 
        source=data)

p5.axis.axis_label=None
p5.axis.visible=False
p5.grid.grid_line_color = None

show(row(p4, p5))

In [44]:
source = ColumnDataSource(df)

hover = HoverTool(tooltips = [("Player", "@Name"), ("Week", "@Week"), ("Points", "@fd_points"), ("Salary", '$'"@fd_salary")])

color_mapper = LinearColorMapper(palette="Viridis256", low=df.fd_salary.min(), high=df.fd_salary.max())

p6 = figure(
    x_axis_label='Salary', 
    y_axis_label='Points', 
    title="POINTS AND SALARY RELATION",
    sizing_mode="stretch_width",
    tools=[hover,'save', 'reset', 'wheel_zoom'],
    toolbar_location='above'
)

p6.circle(
    x='fd_salary', 
    y='fd_points', 
    color=transform('fd_salary', color_mapper), 
    size=10,
    alpha=0.6, 
    source=source)

p6.title.text_font_size='28px'
p6.title.align='center'
p6.title.text_color='#4D5656'
p6.xaxis[0].formatter = NumeralTickFormatter(format="$0")

color_bar = ColorBar(color_mapper=color_mapper, label_standoff=15, location=(0,0), title='Salary')

p6.add_layout(color_bar, 'right')

show(p6)